In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv
import os

# Step 1: Load API key from .env file
load_dotenv()
api_key_01 = os.getenv("HUGGINGFACE_API_KEY")
api_key_02 = os.getenv("GOOGLE_API_KEY")

# Step 2: Select a parser for the output (in this case, a simple string parser)
str_parser = StrOutputParser()

# Step 3: Initialize the HuggingFaceEndpoint with the desired model and task
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key_01
)

# Step 4: Initialize the models for both HuggingFace and Google Generative AI
model_01 = ChatHuggingFace(llm=llm)
model_02 = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.7, api_key = api_key_02)

# Step 5: Create prompt templates for generating notes and quiz questions
prompt_01 = PromptTemplate(
    template='Generate a short notes form the following text: {text}',
    input_variables=["text"]
)

prompt_02 = PromptTemplate(
    template='Generate 5 short questions from the following text: {text}',
    input_variables=["text"]
)

prompt_03 = PromptTemplate(
    template='Merge notes and quiz into a single documnet \n Notes: {notes} \n Quiz: {quiz} .',
    input_variables=["notes", "quiz"]   
)

# Step 6: Create a parallel chain that runs both the notes and quiz generation simultaneously
parallel_chain = RunnableParallel({
    'notes': prompt_01 | model_01 | str_parser,
    'quiz': prompt_02 | model_02 | str_parser
})

# Step 7: Create a merge chain that takes the outputs of the parallel chain and merges them into a single document
merge_chain  = prompt_03 | model_02 | str_parser

# Step 8: Combine the parallel chain and the merge chain into a single chain
chain = parallel_chain | merge_chain

# Step 9: Invoke the chain with a sample text input
text = """# Sequential Chains in LangChain - Detailed Explanation

## Sequential Chain Kya Hota Hai?

**Sequential Chain** ek workflow hai jisme multiple chains ek-ek karke execute hote hain. Pehle chain ka output, doosre chain ka input ban jaata hai.

---

## Sequential vs Simple Chain

| Feature | Simple Chain | Sequential Chain |
|---------|------------|-----------------|
| **Steps** | 1 prompt → 1 model | Multiple prompts → Multiple models |
| **Flow** | Linear (ek hi) | Step-by-step (multiple) |
| **Complexity** | Simple | Advanced |
| **Use Case** | Single task | Multi-step tasks |

---

## Sequential Chain Ka Structure

```
Step 1: Prompt 1 → Model 1 → Parser 1 (Output A)
         ↓
Step 2: Prompt 2 (Input A) → Model 2 → Parser 2 (Output B)
         ↓
Step 3: Prompt 3 (Input B) → Model 3 → Parser 3 (Final Output)
```

---

## Real-World Example

**Task:** Kisi book ke baare mein detailed summary likho

**Sequential Steps:**

1️⃣ **Step 1**: Book ke baare mein basic info generate karo  
2️⃣ **Step 2**: Us info se detailed summary likho  
3️⃣ **Step 3**: Summary se key takeaways nikalo  

---

## Code Example - Sequential Chain

````python
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

# Setup
load_dotenv()
api_key = os.getenv("HUGGINGFACE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key,
    temperature=0.3,
    max_new_tokens=200
)

model = ChatHuggingFace(llm=llm)
str_parser = StrOutputParser()

# ============ STEP 1: Generate Book Info ============
prompt_1 = PromptTemplate(
    template="Provide basic information about the book '{book_name}' in 3 lines. Include author, genre, and publication year.",
    input_variables=["book_name"]
)

chain_1 = prompt_1 | model | str_parser

# ============ STEP 2: Generate Summary ============
prompt_2 = PromptTemplate(
    template="Based on this book information: {book_info}\n\nNow write a detailed summary of the book in 5 points.",
    input_variables=["book_info"]
)

chain_2 = prompt_2 | model | str_parser

# ============ STEP 3: Extract Key Takeaways ============
prompt_3 = PromptTemplate(
    template="From this summary: {summary}\n\nExtract 3 most important key takeaways from the book.",
    input_variables=["summary"]
)

chain_3 = prompt_3 | model | str_parser

# ============ EXECUTE SEQUENTIAL CHAIN ============
print("=== STEP 1: Book Information ===")
book_info = chain_1.invoke({"book_name": "1984"})
print(book_info)

print("\n=== STEP 2: Detailed Summary ===")
summary = chain_2.invoke({"book_info": book_info})
print(summary)

print("\n=== STEP 3: Key Takeaways ===")
takeaways = chain_3.invoke({"summary": summary})
print(takeaways)
````

---

## Using RunnableSequence (Advanced)

````python
from langchain_core.runnables import RunnablePassthrough

# Method 1: Manual Sequential Execution (Recommended for clarity)
def execute_sequential(book_name):
    info = chain_1.invoke({"book_name": book_name})
    summary = chain_2.invoke({"book_info": info})
    takeaways = chain_3.invoke({"summary": summary})
    
    return {
        "book_info": info,
        "summary": summary,
        "takeaways": takeaways
    }

result = execute_sequential("To Kill a Mockingbird")
print(result)
````

---

## LangChain's SequentialChain (Older Method)

````python
from langchain.chains import SequentialChain

# Define chains with memory
chain_1 = LLMChain(llm=model, prompt=prompt_1, output_key="book_info")
chain_2 = LLMChain(llm=model, prompt=prompt_2, output_key="summary")
chain_3 = LLMChain(llm=model, prompt=prompt_3, output_key="takeaways")

# Create sequential chain
overall_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3],
    input_variables=["book_name"],
    output_variables=["book_info", "summary", "takeaways"],
    verbose=True
)

result = overall_chain({"book_name": "1984"})
````

---

## Real Example: Article Writing Workflow

````python
# STEP 1: Generate Title
prompt_title = PromptTemplate(
    template="Generate an engaging article title about {topic}.",
    input_variables=["topic"]
)
chain_title = prompt_title | model | str_parser

# STEP 2: Generate Outline
prompt_outline = PromptTemplate(
    template="Given the title '{title}', create a 5-point article outline.",
    input_variables=["title"]
)
chain_outline = prompt_outline | model | str_parser

# STEP 3: Write Full Article
prompt_article = PromptTemplate(
    template="Using this outline:\n{outline}\n\nWrite a detailed article.",
    input_variables=["outline"]
)
chain_article = prompt_article | model | str_parser

# EXECUTE
print("Step 1: Generating Title...")
title = chain_title.invoke({"topic": "Artificial Intelligence"})

print("Step 2: Generating Outline...")
outline = chain_outline.invoke({"title": title})

print("Step 3: Writing Article...")
article = chain_article.invoke({"outline": outline})

print("\n=== FINAL ARTICLE ===")
print(article)
````

---

## Sequential Chain Flow Diagram

```
INPUT: User Request
    ↓
STEP 1: Process Input
    ├─ Prompt Template
    ├─ LLM Model
    └─ Output Parser → Output A
    ↓
STEP 2: Use Output A as Input
    ├─ Prompt Template (with Output A)
    ├─ LLM Model
    └─ Output Parser → Output B
    ↓
STEP 3: Use Output B as Input
    ├─ Prompt Template (with Output B)
    ├─ LLM Model
    └─ Output Parser → Final Output
    ↓
OUTPUT: Final Result
```

---

## Key Advantages

✅ **Complex Tasks** - Multi-step workflows ko handle karta hai  
✅ **Reusable** - Har step independently use ho sakta hai  
✅ **Debugging** - Har step ka output dekhna aasan  
✅ **Flexible** - Output of one step → Input of next  
✅ **Professional** - Production-level applications  

---

## Common Use Cases

1. **Content Generation**
   - Title → Outline → Full Article

2. **Question Answering**
   - Extract Context → Understand Question → Generate Answer

3. **Code Generation**
   - Understand Request → Plan Solution → Generate Code

4. **Research**
   - Search Topic → Summarize → Create Report

5. **Translation + Summarization**
   - Translate Text → Summarize → Extract Key Points

---

## Error Handling in Sequential Chains

````python
def execute_with_error_handling(book_name):
    try:
        print("Step 1: Fetching book info...")
        info = chain_1.invoke({"book_name": book_name})
        if not info:
            raise ValueError("Step 1 failed: No book info generated")
        
        print("Step 2: Generating summary...")
        summary = chain_2.invoke({"book_info": info})
        if not summary:
            raise ValueError("Step 2 failed: No summary generated")
        
        print("Step 3: Extracting takeaways...")
        takeaways = chain_3.invoke({"summary": summary})
        
        return {
            "status": "success",
            "book_info": info,
            "summary": summary,
            "takeaways": takeaways
        }
    
    except Exception as e:
        return {
            "status": "error",
            "error_message": str(e)
        }

result = execute_with_error_handling("1984")
print(result)
````

---

## Key Takeaways

🔹 **Sequential = Step-by-step execution**  
🔹 **Output of Step N → Input of Step N+1**  
🔹 **Multiple prompts, multiple models**  
🔹 **Excellent for complex workflows**  
🔹 **Easy debugging and monitoring**  

**Bas Itna Samajh Loo: Sequential Chain = Pehla chain → Doosra chain → Teesra chain** ✅"""

# Step 10: Invoke the chain with the sample text
result = chain.invoke({'text': text})

# Step 11: Print the final response
print(result)


Here's the merged document combining your notes and quiz about Sequential Chains:

---

# Sequential Chain Ka Samajh: Notes & Quiz

## Understanding Sequential Chains

1.  **Sequential chain** ek workflow hai jo multiple chains ko ek-ek karke execute karta hai.
2.  **Pehla chain ka output** doosre chain ka input ban jata hai.
3.  **Iska sahi se upyog** complex tasks, reusable code, debugging, flexible workflows, aur professional applications ke liye hai.

### Key Features

*   **Steps**: Multiple prompts, multiple models
*   **Flow**: Step-by-step
*   **Complexity**: Advanced
*   **Use Case**: Multi-step tasks

### Real-World Example

*   **Task**: Book ke baare mein detailed summary likho.
*   **Steps**:
    1.  **Step 1**: Book ke baare mein basic info generate karo.
    2.  **Step 2**: Us info se detailed summary likho.
    3.  **Step 3**: Summary se key takeaways nikalo.

### Code Example: Implementing a Sequential Flow

```python
from langchain_huggingface import ChatHuggingFace, 

In [4]:
chain.get_graph().print_ascii()

               +---------------------------+                     
               | Parallel<notes,quiz>Input |                     
               +---------------------------+                     
                  ****                 ***                       
               ***                        ****                   
             **                               **                 
+----------------+                       +----------------+      
| PromptTemplate |                       | PromptTemplate |      
+----------------+                       +----------------+      
          *                                       *              
          *                                       *              
          *                                       *              
+-----------------+                  +------------------------+  
| ChatHuggingFace |                  | ChatGoogleGenerativeAI |  
+-----------------+                  +------------------------+  
          